In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import copy

## Парсинг amc

In [2]:
def read_frame(frame, frames_dict):
    # parse joints
    for joint in frame:
        joint_info = joint.strip().split()
        joint_name = joint_info[0]
        joint_info.remove(joint_name)
        
        for i in range(len(joint_info)):
            coord = joint_info[i]
            frames_dict[joint_name + '_' + str(i)] = float(coord)
            
    return frames_dict

In [3]:
def parse_amc(filepath):
    with open(filepath) as f:
        content = f.read().splitlines()

    for idx, line in enumerate(content):
        if line == ':DEGREES':
            content = content[idx+1:]
            break
            
    # get idx of frames beginning
    frame_beginning = []
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if len(splitted_line) == 1:
            frame_beginning.append(idx)
    # parse frames
    all_frames = []
    for i in range(len(frame_beginning)-1):
        frame = content[frame_beginning[i]:frame_beginning[i+1]]
        frames_dict = {}
        frames_dict['frame_number'] = frame[0]
        # parse joints
        frames_dict = read_frame(frame[1:], frames_dict)
        all_frames.append(frames_dict)

    all_data = pd.DataFrame(all_frames)            
        
    return all_data

In [4]:
content = parse_amc('../data/02_01.amc')

In [5]:
content.head()

,frame_number,head_0,head_1,head_2,lclavicle_0,lclavicle_1,lfemur_0,lfemur_1,lfemur_2,lfingers_0,...,rwrist_0,thorax_0,thorax_1,thorax_2,upperback_0,upperback_1,upperback_2,upperneck_0,upperneck_1,upperneck_2
0,1,2.97226,2.54741,-1.92752,-1.620100e-14,-1.431250e-14,-28.0191,-1.020780,-20.1783,7.12502,...,-16.7798,-0.599593,1.31944,-2.05527,3.47255,2.55226,1.65684,1.617840,5.61246,-6.40733
1,2,3.01316,2.59206,-1.92353,-3.061280e-14,-7.951390e-16,-27.7269,-0.953135,-20.5822,7.12502,...,-16.5037,0.078456,1.35992,-1.86912,3.91254,2.58553,1.67171,1.419540,5.71695,-6.43796
2,3,2.90215,2.58200,-1.86960,3.975690e-15,-1.033680e-14,-27.6281,-0.810148,-21.0041,7.12502,...,-16.2262,0.211031,1.41449,-1.83734,3.96845,2.68906,1.61541,1.081860,5.66983,-6.25219
3,4,2.73437,2.55894,-1.86943,1.376580e-14,0.000000e+00,-27.6637,-0.391122,-21.6105,7.12502,...,-16.0827,0.127573,1.45150,-1.90802,3.81888,2.77415,1.44970,0.706102,5.58519,-6.09066
4,5,2.59153,2.57436,-1.87971,-2.042510e-14,-7.156250e-15,-27.6786,-0.161279,-21.8039,7.12502,...,-16.0025,0.158931,1.46518,-1.93228,3.70601,2.80336,1.40701,0.388960,5.58739,-5.98980


In [6]:
content.shape

(342, 63)

## Парсинг asf

In [122]:
def is_float(s):
    
    try:
        float(s)
        return float(s)
    
    except ValueError:
        return False
    
def is_number(s):
    
    if s.isdigit() == False:
        float_s = is_float(s)
        if float_s == False:
            val = False
        else:
            val = float_s
    else:
        val = int(s)
    return val
        
 
    

def read_bone_params(bone_data):
    
    clear_bone_data = [line for line in bone_data if ('end' in line)|('id' in line)|('begin' in line) == False] 
    
    # merge in one line all "limits"
    for id, param in enumerate(clear_bone_data):
        if param.strip().split()[0] == 'limits':
            clear_bone_data_new = clear_bone_data[:id]
            new_limits = "".join(clear_bone_data[id:])
            clear_bone_data_new.append(new_limits)
            break
        else:
            clear_bone_data_new = clear_bone_data  
    
    params_dict = {}
    for line in clear_bone_data_new:
        str_data = line.strip().split()
        if str_data[0] == 'name':
            bone_name = str_data[1]   
        else:
            values = list(str_data[1:])
            values_new = []
            for s in values:
                par = is_number(s)                             
                if  type(par) != bool:
                    val = par
                else:
                    val = s
                values_new.append(val)    
            params_dict[str_data[0]] = values_new
            
    bone_params = {}
    bone_params[bone_name] = params_dict   
    
    return bone_params


def parse_asf(filepath):

    with open(filepath) as f:
        content = f.read().splitlines()

    
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if line.rfind('hierarchy') > -1:
            hierarchy_start=idx
        if line.rfind('bonedata') > -1:
            bonedata_start=idx
            
    # считываем нижний блок с расписанной архитектурой   
    hierarchy_content = content[hierarchy_start:]
    clear_hierarchy = []
    for idx, line in enumerate(hierarchy_content):
        if (('hierarchy' in line) | ('begin' in line) | ('end' in line)) == False:
            clear_hierarchy.append(line.strip())

    hierarchy_dict = {}        
    for line in clear_hierarchy:
        splitted_line = line.split()
        for i in range(len(splitted_line)-1):
            hierarchy_dict[splitted_line[i+1]] = splitted_line[0]
        
        
    # надо теперь посчитать сколько bones
    bone_types = []
    bone_types_id = []
    for idx, line in enumerate(content):
        if (line.rfind('name') > -1 and idx > bonedata_start):
            bone_types_id.append(idx)
            # надо теперь посчитать сколько bones
            splitted_line = line.strip().split()
            #но и сохраняем
            bone_types.append(splitted_line[1])
            
    bone_types_iter = copy.deepcopy(bone_types_id)
    bone_types_iter.append(hierarchy_start)
    all_params = {}

    for i in range(len(bone_types_iter)-1):
        bone_data = content[bone_types_iter[i]:bone_types_iter[i+1]]
        bone_params = read_bone_params(bone_data)
        all_params = dict(all_params, **bone_params)
        
    return all_params, hierarchy_dict

In [123]:
filepath = '../data/02.asf'
all_p, hierarchy_dict = parse_asf(filepath)

In [176]:
hierarchy_dict

{'lhipjoint': 'root',
 'rhipjoint': 'root',
 'lowerback': 'root',
 'lfemur': 'lhipjoint',
 'ltibia': 'lfemur',
 'lfoot': 'ltibia',
 'ltoes': 'lfoot',
 'rfemur': 'rhipjoint',
 'rtibia': 'rfemur',
 'rfoot': 'rtibia',
 'rtoes': 'rfoot',
 'upperback': 'lowerback',
 'thorax': 'upperback',
 'lowerneck': 'thorax',
 'lclavicle': 'thorax',
 'rclavicle': 'thorax',
 'upperneck': 'lowerneck',
 'head': 'upperneck',
 'lhumerus': 'lclavicle',
 'lradius': 'lhumerus',
 'lwrist': 'lradius',
 'lhand': 'lwrist',
 'lthumb': 'lwrist',
 'lfingers': 'lhand',
 'rhumerus': 'rclavicle',
 'rradius': 'rhumerus',
 'rwrist': 'rradius',
 'rhand': 'rwrist',
 'rthumb': 'rwrist',
 'rfingers': 'rhand'}

In [7]:
all_p

{'lhipjoint': {'direction': [0.655637, -0.713449, 0.247245],
  'length': [2.52691],
  'axis': [0, 0, 0, 'XYZ']},
 'lfemur': {'direction': [0.34202, -0.939693, 0],
  'length': [7.59371],
  'axis': [0, 0, 20, 'XYZ'],
  'dof': ['rx', 'ry', 'rz'],
  'limits': ['(-160.0', '20.0)', '(-70.0', '70.0)', '(-60.0', '70.0)']},
 'ltibia': {'direction': [0.34202, -0.939693, 0],
  'length': [7.28717],
  'axis': [0, 0, 20, 'XYZ'],
  'dof': ['rx'],
  'limits': ['(-10.0', '170.0)']},
 'lfoot': {'direction': [0.0886837, -0.243657, 0.965798],
  'length': [2.2218],
  'axis': [-90.0, 7.62852e-16, 20, 'XYZ'],
  'dof': ['rx', 'rz'],
  'limits': ['(-45.0', '90.0)', '(-70.0', '20.0)']},
 'ltoes': {'direction': [1.53547e-11, -4.22004e-11, 1],
  'length': [1.11249],
  'axis': [-90.0, 7.62852e-16, 20, 'XYZ'],
  'dof': ['rx'],
  'limits': ['(-90.0', '20.0)']},
 'rhipjoint': {'direction': [-0.645062, -0.722004, 0.250209],
  'length': [2.49697],
  'axis': [0, 0, 0, 'XYZ']},
 'rfemur': {'direction': [-0.34202, -0.9396

In [174]:
trans_dict = {}
trans_dict['root'] = np.array([0, 0, 0]).reshape(3, 1)

def get_transform(all_p, trans_dict, child_name='lowerback', parent_name='root'):
    
    par_coor = trans_dict[parent_name]
#     print(par_coor)
    # get child data
    direct = all_p[child_name]['direction']
    length = all_p[child_name]['length']
    vect = np.array([length[0], 0, 0])
#     print(vect)
    # тут надо вектор столбец сделать
    a = direct[0] * np.pi / 180 #угол 20 град.
    b = direct[1] * np.pi / 180 #угол 20 град.
    c = direct[2] * np.pi / 180 #угол 20 град.

    Ax = np.array([[1,0,0], [0,np.cos(a),-np.sin(a)], [0,np.sin(a),np.cos(a)]])
    Ay = np.array([[np.cos(b),0,np.sin(b)], [0,1,0], [-np.sin(b),0,np.cos(b)]])
    Az = np.array([[np.cos(c),-np.sin(c),0], [np.sin(c),np.cos(c),0], [0,0,1]])
    S = np.dot(np.dot(Ax, Ay), Az)
    vect_in_par = np.dot(vect, S).reshape(3, 1)
#     print(vect_in_par)
    vect_in_root = vect_in_par + par_coor
    
    return vect_in_root

def get_child_transform(current_bone, hierarchy_dict, all_p, trans_dict):
    
    childs = []
    for key, value in hierarchy_dict.items():
        if value == current_bone:
            childs.append(key)

    for child in childs:
        child_coord = get_transform(all_p, trans_dict, child_name=child, parent_name=current_bone)
        trans_dict[child] = child_coord
        
    return trans_dict

# for root
trans_dict = get_child_transform(current_bone='root', hierarchy_dict=hierarchy_dict, all_p=all_p, trans_dict=trans_dict)

for current, parent in hierarchy_dict.items():
#     print(current)
    transformed_bones = list(trans_dict.keys())
    if (current not in transformed_bones) and (parent in transformed_bones):  
        trans_dict = get_child_transform(current_bone=parent, hierarchy_dict=hierarchy_dict, 
                                             all_p=all_p, trans_dict=trans_dict)

In [175]:
trans_dict

{'root': array([[0],
        [0],
        [0]]), 'lhipjoint': array([[ 2.52669057],
        [-0.01090334],
        [-0.03146436]]), 'rhipjoint': array([[ 2.49674794],
        [-0.01090329],
        [-0.03146435]]), 'lowerback': array([[2.05911637],
        [0.00246256],
        [0.0358559 ]]), 'lfemur': array([[10.1193793 ],
        [-0.01090334],
        [-0.15600121]]), 'ltibia': array([[ 1.74055693e+01],
        [-1.09033422e-02],
        [-2.75510812e-01]]), 'lfoot': array([[19.62703353],
        [-0.04835268],
        [-0.28495925]]), 'ltoes': array([[20.73935409],
        [-0.06776831],
        [-0.28495925]]), 'rfemur': array([[10.08306753],
        [-0.01090329],
        [-0.15589674]]), 'rtibia': array([[ 1.72974771e+01],
        [-1.09032948e-02],
        [-2.74228984e-01]]), 'rfoot': array([[19.53448999],
        [-0.04814079],
        [-0.28526949]]), 'rtoes': array([[20.65001007],
        [-0.06761226],
        [-0.28526949]]), 'upperback': array([[4.12403183e+00],
       

In [ ]:
read units (defult length)

### draw data from asf

In [25]:
import plotly.plotly as py
import plotly.graph_objs as go

In [197]:
value_array

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.52669057e+00, -1.09033422e-02, -3.14643564e-02],
       [ 2.49674794e+00, -1.09032948e-02, -3.14643527e-02],
       [ 2.05911637e+00,  2.46255872e-03,  3.58559010e-02],
       [ 1.01193793e+01, -1.09033422e-02, -1.56001212e-01],
       [ 1.74055693e+01, -1.09033422e-02, -2.75510812e-01],
       [ 1.96270335e+01, -4.83526796e-02, -2.84959247e-01],
       [ 2.07393541e+01, -6.77683072e-02, -2.84959247e-01],
       [ 1.00830675e+01, -1.09032948e-02, -1.55896741e-01],
       [ 1.72974771e+01, -1.09032948e-02, -2.74228984e-01],
       [ 1.95344900e+01, -4.81407877e-02, -2.85269493e-01],
       [ 2.06500101e+01, -6.76122630e-02, -2.85269493e-01],
       [ 4.12403183e+00,  3.49581698e-03,  7.18838897e-02],
       [ 6.19178685e+00,  3.35557353e-03,  1.07976399e-01],
       [ 7.76580709e+00,  7.43505042e-04,  1.35326275e-01],
       [ 9.85155156e+00,  6.38609539e-03,  1.23760448e-01],
       [ 9.78619788e+00,  9.04802030e-03

In [204]:
bone_list = []
values = []
value_array = np.zeros(shape=(len(trans_dict), 3))

for idx, (key, value) in enumerate(trans_dict.items()):
    bone_list.append(key)
    value_array[idx] = value.reshape(len(value))
    
# plot
trace1 = go.Scatter3d(
    x=value_array[:, 0],
    y=value_array[:, 1],
    z=value_array[:, 2],
    mode='markers+text',
    text=bone_list
)
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=10, range = [-1,22],),
                    yaxis = dict(
                        nticks=10, range = [-1,22],),
                    zaxis = dict(
                        nticks=10, range = [-1,22],),)
#                     width=700,
#                     margin=dict(
#                     r=20, l=10,
#                     b=10, t=10)
                  )
py.iplot(go.Figure(data=[trace1], layout=layout), filename='3d-scatter-colorscale')

In [203]:
np.min(value_array)

-0.2852694930955815

In [195]:
value_array   

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.52669057e+00, -1.09033422e-02, -3.14643564e-02],
       [ 2.49674794e+00, -1.09032948e-02, -3.14643527e-02],
       [ 2.05911637e+00,  2.46255872e-03,  3.58559010e-02],
       [ 1.01193793e+01, -1.09033422e-02, -1.56001212e-01],
       [ 1.74055693e+01, -1.09033422e-02, -2.75510812e-01],
       [ 1.96270335e+01, -4.83526796e-02, -2.84959247e-01],
       [ 2.07393541e+01, -6.77683072e-02, -2.84959247e-01],
       [ 1.00830675e+01, -1.09032948e-02, -1.55896741e-01],
       [ 1.72974771e+01, -1.09032948e-02, -2.74228984e-01],
       [ 1.95344900e+01, -4.81407877e-02, -2.85269493e-01],
       [ 2.06500101e+01, -6.76122630e-02, -2.85269493e-01],
       [ 4.12403183e+00,  3.49581698e-03,  7.18838897e-02],
       [ 6.19178685e+00,  3.35557353e-03,  1.07976399e-01],
       [ 7.76580709e+00,  7.43505042e-04,  1.35326275e-01],
       [ 9.85155156e+00,  6.38609539e-03,  1.23760448e-01],
       [ 9.78619788e+00,  9.04802030e-03